This notebook runs a number of queries to check on the status of the TIR database.

In [1]:
import requests,configparser

In [2]:
# Get API keys and any other config details from a file that is external to the code.
config = configparser.RawConfigParser()
config.read_file(open(r'../config/stuff.py'))

In [3]:
# Build base URL with API key using input from the external config.
def getBaseURL():
    gc2APIKey = config.get('apiKeys','apiKey_GC2_BCB').replace('"','')
    apiBaseURL = "https://gc2.mapcentia.com/api/v1/sql/bcb?key="+gc2APIKey
    return apiBaseURL

In [19]:
countITISExact = requests.get(getBaseURL()+"&q=SELECT COUNT(*) AS num FROM tir.tir2 WHERE itis->'itisMatchMethod' LIKE 'ExactMatch%'").json()
print ("Exact ITIS Matches: "+str(countITISExact["features"][0]["properties"]["num"]))

countITISFuzzy = requests.get(getBaseURL()+"&q=SELECT COUNT(*) AS num FROM tir.tir2 WHERE itis->'itisMatchMethod' LIKE 'FuzzyMatch%'").json()
print ("Fuzzy ITIS Matches: "+str(countITISFuzzy["features"][0]["properties"]["num"]))

countITISNot = requests.get(getBaseURL()+"&q=SELECT COUNT(*) AS num FROM tir.tir2 WHERE itis->'itisMatchMethod' LIKE 'NotMatched%'").json()
print ("ITIS Not Matched: "+str(countITISNot["features"][0]["properties"]["num"]))

Exact ITIS Matches: 13453
Fuzzy ITIS Matches: 774
ITIS Not Matched: 1915


In [20]:
countByRank = requests.get(getBaseURL()+"&q=SELECT COUNT(*) as numrank, itis -> 'rank' AS rank FROM tir.tir2 WHERE itis -> 'itisMatchMethod' NOT LIKE 'NotMatched%' GROUP BY itis -> 'rank'").json()
for feature in countByRank["features"]:
    print (feature["properties"]["rank"]+" - "+str(feature["properties"]["numrank"]))

Class - 2
Family - 158
Genus - 49
Order - 22
Phylum - 3
Species - 12449
Subclass - 1
Subfamily - 1
Suborder - 2
Subspecies - 1176
Variety - 376


In [6]:
itisNotMatched = requests.get(getBaseURL()+"&q=SELECT itis -> 'itisMatchMethod' AS notmatched FROM tir.tir2 WHERE itis -> 'itisMatchMethod' LIKE 'NotMatched%' ORDER BY itis -> 'matchMethod'").json()
for feature in itisNotMatched["features"]:
    print (feature["properties"]["notmatched"])

NotMatched:Actinopyga mauritiana
NotMatched:Potamilus alatus
NotMatched:Polygonum sawatchense ssp. sawatchense g4g5tnrs1s3
NotMatched:Artace cribraria
NotMatched:Aspidoscelis burti
NotMatched:Aspidoscelis exsanguis
NotMatched:Aspidoscelis flagellicauda
NotMatched:Aspidoscelis xanthonota
NotMatched:Asterias forbesii
NotMatched:Stauroneus maunakeensis
NotMatched:Birgus latro
NotMatched:Borodinia missouriensis
NotMatched:Pyganodon subgibbosa
NotMatched:Caecidotea sp. 1
NotMatched:Caecidotea sp. 8
NotMatched:Calidris subruficollis
NotMatched:Cicinnus melsheimeri
NotMatched:Eutamias cinereicollis
NotMatched:Deschampsia flexuosa
NotMatched:Desmognathus cf. conanti
NotMatched:Diapensia lapponica ssp. lapponica
NotMatched:Euthyonidiella trita
NotMatched:Faronta rubripennis
NotMatched:Flabellorhagidia pecki
NotMatched:Eutrichapion huron
NotMatched:Mysella sp. c
NotMatched:Gracilaria abbottiana
NotMatched:Havelockia inermis
NotMatched:Pyrgulopsis anatina
NotMatched:Flexamia delongi
NotMatched:Fl

In [21]:
countByFWSListingStatus = requests.get(getBaseURL()+"&q=SELECT COUNT(*) as numstatus, tess -> 'StatusText' AS status FROM tir.tir2 WHERE tess -> 'result' <> 'none' GROUP BY tess -> 'StatusText'").json()
for feature in countByFWSListingStatus["features"]:
    print (feature["properties"]["status"]+" - "+str(feature["properties"]["numstatus"]))

Endangered - 36
Status Undefined - 3
Resolved Taxon - 26
Species of Concern - 51
Threatened - 12
Original Data in Error - Erroneous Data - 2
Candidate - 1
Under Review in the Candidate or Petition Process - 23
Experimental Population, Non-Essential - 1
Recovery - 2
